In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp
from sklearn.covariance import LedoitWolf

# Load price data (Assume 'prices.csv' with Date as index)
df = pd.read_csv("/content/price_data no rfr.csv", index_col=0, parse_dates=True)

# Compute simple returns (percentage change)
returns = df.pct_change().dropna()

# Estimate Ledoit-Wolf covariance matrix
lw = LedoitWolf()
cov_matrix = lw.fit(returns).covariance_

# Mean returns (Ensure it's a NumPy array)
expected_returns = returns.mean().values.reshape(-1, 1)  # Daily expected returns as column vector

# Number of assets
n_assets = len(expected_returns)

# Define optimization variables
w = cp.Variable((n_assets, 1))  # Asset weights as a column vector

# Mean-variance optimization: minimize risk (variance) adjusted by risk_aversion and return
risk_aversion = 5  # Adjust as needed

portfolio_variance = cp.quad_form(w, cov_matrix)
expected_portfolio_return = cp.matmul(expected_returns.T, w)

objective = cp.Minimize(risk_aversion * portfolio_variance - expected_portfolio_return)
constraints = [
    cp.sum(w) == 1,  # Weights sum to 1
    w >= 0.02,          # No short-selling
    w <= 0.4              # Maximum 35% allocation per asset
]

# Solve the optimization problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Get optimal weights
optimal_weights = w.value.flatten()

# Annualize returns and volatility
trading_days = 252  # Typical number of trading days in a year

# Calculate daily portfolio expected return and volatility
daily_expected_return = np.dot(optimal_weights, expected_returns.flatten())
daily_variance = np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))
daily_volatility = np.sqrt(daily_variance)

# Annualization
annual_expected_return = daily_expected_return * trading_days
annual_volatility = daily_volatility * np.sqrt(trading_days)

# Display results
portfolio_allocation = pd.DataFrame({'Asset': df.columns, 'Weight': optimal_weights})
print(portfolio_allocation)
print(f"\nAnnualized Expected Portfolio Return: {annual_expected_return:.4f}")
print(f"Annualized Portfolio Volatility: {annual_volatility:.4f}")


   Asset    Weight
0   ACWX  0.020000
1    AGG  0.173176
2   BNDX  0.400000
3    TLT  0.020000
4  ^GSPC  0.386824

Annualized Expected Portfolio Return: 0.0645
Annualized Portfolio Volatility: 0.0827
